In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


In [ ]:
# Configuration parameters
CONFIG = {
    'IMG_SIZE': (32, 32),
    'BATCH_SIZE': 64,
    'NUM_CLASSES': 43,
    'EPOCHS': 50,
    'LEARNING_RATE': 0.001,
    'RANDOM_STATE': 42,
    'TEST_SIZE': 0.2
}

In [ ]:
# Load and preprocess data
def load_data(dataset_path):
    df = pd.read_csv(os.path.join(dataset_path, "Train.csv"))
    image_paths = [os.path.join(dataset_path, "Train", str(row['ClassId']), row['Path']) 
                  for _, row in df.iterrows()]
    
    # Load and preprocess images
    images = [cv2.resize(cv2.imread(img_path), CONFIG['IMG_SIZE']) 
             for img_path in image_paths]
    images = np.array(images) / 255.0
    
    return images, df["ClassId"].values

In [ ]:
# Build the model
def create_model():
    model = keras.Sequential([
        # First Conv Block
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Second Conv Block
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Third Conv Block
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Dense Layers
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(CONFIG['NUM_CLASSES'], activation='softmax')
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=CONFIG['LEARNING_RATE']),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
# Plot training history
def plot_training_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    ax1.plot(history.history['accuracy'], label='Training')
    ax1.plot(history.history['val_accuracy'], label='Validation')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    
    ax2.plot(history.history['loss'], label='Training')
    ax2.plot(history.history['val_loss'], label='Validation')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Load and prepare data
DATASET_PATH = "path_to_dataset"
images, labels = load_data(DATASET_PATH)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, 
    test_size=CONFIG['TEST_SIZE'], 
    random_state=CONFIG['RANDOM_STATE']
)

# Convert labels to categorical
y_train = keras.utils.to_categorical(y_train, num_classes=CONFIG['NUM_CLASSES'])
y_test = keras.utils.to_categorical(y_test, num_classes=CONFIG['NUM_CLASSES'])

# Create and compile model
model = create_model()
model.summary()

# Define callbacks
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7)
]

# Train model
history = model.fit(
    X_train, y_train,
    batch_size=CONFIG['BATCH_SIZE'],
    epochs=CONFIG['EPOCHS'],validation_split=0.2,
    callbacks=callbacks
)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")